In [126]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

def earnings_releases(ticker, market_cap, sector):
    #print("Looking up ticker {}".format(ticker))
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:46.0) '\
        'Gecko/20100101 Firefox/46.0'
    headers = {'user-agent': user_agent}
    base_url = 'http://www.streetinsider.com/ec_earnings.php?q={}'\
        .format(ticker)
    s = requests.Session()
    a = requests.adapters.HTTPAdapter(max_retries=0)
    s.mount('http://', a)
    
    html = str(s.get(base_url, headers=headers).content)
    soup = BeautifulSoup(html, 'lxml')
    
    # There are two tables - one is the more recent earnings,
    # second is historical
    tables = pd.read_html(str(soup.find_all('table')[1:3]))
    
    table = pd.concat(tables)
    table.columns = table.iloc[0]
    table = table.iloc[1:][['Date', 'EPS', 'Cons.', 'Surprise', 'Revs']]
    
    def parse_float(series):
        try:
            return (series.str.replace('$', '')
                    .str.replace('+', '')
                    .str.replace('[A-Za-z]', '')
                    .astype(float))
        except (ValueError, AttributeError):
            return [np.nan] * len(series)
    
    table['Surprise'] = parse_float(table['Surprise'])
    table['EPS'] = parse_float(table['EPS'])
    
    table['Ticker'] = [ticker] * len(table)
    table['Market Cap'] = [market_cap] * len(table)
    table['Sector'] = [sector] * len(table)
    table['BeforeOpen'] = ['Before Open' in html] * len(table)
    return table.dropna()

In [98]:
r3k_raw = pd.read_csv('https://www.ishares.com/us/products/239714/'
                  'ishares-russell-3000-etf/1467271812596.ajax'
                  '?fileType=csv&fileName=IWV_holdings&dataType=fund', header=10)

r3k = r3k_raw[r3k_raw['Asset Class'] == 'Equity']

In [130]:
e = [earnings_releases(x['Ticker'], x['Notional Value'], x['Sector']) for i, x in r3k.iterrows()]
all_earnings = pd.concat(e)
all_earnings.to_csv('all_earnings.csv')